In [32]:
import numpy as np
from scipy.interpolate import PchipInterpolator
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
import warnings
import pandas as pd
import altair as alt
warnings.filterwarnings('ignore')
output_notebook()
alt.data_transformers.disable_max_rows()

Loading BokehJS ...

DataTransformerRegistry.enable('default')

## Import the data 

In [33]:
data = np.loadtxt('data/PT_wafer2_after_d_diodes.dat', delimiter=',')
lendata = data.shape[0] - 1
angles = (np.linspace(0, (lendata - 1), lendata)) * (data[-1, 29] - data[-1, 28]) * (2 * np.pi / lendata / 360) + (data[-1, 28] + 9.5 + 180) * np.pi / 180
radii = (np.linspace(0, (lendata-1), lendata)) * (data[-1,31]-data[-1,30])/lendata + data[-1, 30]

Aq = []
M = []
intens = []
x = np.arange(1, 33)
datamat = data[:lendata,:] - 1

## Calcute Aq, M and I values

In [34]:
import time
start_time = time.time()

sx = np.arange(1, 32, 0.2)
for indx in range(lendata):
    # a. Do PChip spline  interpolation
    diodes = datamat[indx,:]
    pchip = PchipInterpolator(x, diodes)
    sy = pchip(sx)
    sy = 100 * sy / np.max(sy)
    c = []

    # b. Concatenation/histogram
    for indx2 in range(len(sx)):
        c.extend([indx2+1]*round(sy[indx2]))

    stddev = np.std(c) / 5
    Aq.append(1.02 * np.exp(1.987 * np.log(stddev) + 0.16))
    M.append((np.mean(c) - 1) / 5 - 15.5)
    intens.append(np.sum(sy))

## Plot

### Get x/y coordinates

In [36]:
# 1. Get x and y coordinates
xvals = radii * np.cos(angles) * 1e-3
yvals = radii * np.sin(angles) * 1e-3
zvals = Aq

# 2. Clamp values to max of 3
zvalscut = 3
zvalsnew = np.array(zvals)
zvalsnew[zvalsnew > zvalscut] = zvalscut

# 3. Make a df
index = np.arange(len(xvals))
df = pd.DataFrame({
    'index':index,
    'x': xvals,
    'y': yvals,
    'z': zvalsnew,   
})

df.to_csv('data/test.csv', index=False)

### Generate plot

In [37]:
#| column: screen-inset-right
# 4. Rank_text
interval = alt.selection_interval()
rank_text = alt.Chart(df).mark_text(dx=20, dy=-5, align='left').encode(
    x=alt.value(10),  # x position of the text
    y=alt.value(10),  # y position of the text
    text=alt.condition(
        interval,
        alt.Text('index:Q', format='.2f'),  # display 'x' column for selected points
        alt.value('')  # display empty string for unselected points
    ),
    color=alt.value('black'), # color of the text
    fontSize = 5
)

# 5. Wafer plot
wafer_plot_b = alt.Chart(df).mark_circle().encode(
    x='x:Q', y='y:Q',
    color=alt.condition(
        interval,
        alt.Color('z:Q', scale=alt.Scale(scheme='turbo')),
        alt.value('lightgray')
    ),
    tooltip=['x', 'y', 'z', 'index']
).properties(height=600, width=600).add_selection(interval)

# create a table with the selected points
selected_points_table = alt.Chart(df).transform_filter(
    interval
).mark_text().encode(
    x=alt.value(0),
    y=alt.Y('row_number:O', axis=None),
    text='index:Q',
).transform_window(
    row_number='row_number()'
).properties(
    height=600,
    width=50
)

# vertically concatenate the two charts
alt.hconcat(wafer_plot_b, selected_points_table)



alt.HConcatChart(...)